<a href="https://colab.research.google.com/github/AmandaOnData/tutorial_GPT_letter_class/blob/main/AI_Tailored_letter_Amanda_Williamson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-Tailored Cover Letter
This is for educational use only, and not intended as advice or a replacement for writing cover letters. Course: Leading for an Uncertain Future, University of Waikato, 2023. Week 9. **Dr Amanda J Williamson @AmandaOnData**

---



In this tutorial you will learn how to create a tailored CV using artificial intelligence. Specifically it uses OpenAI's GPT 3.5 model to help craft a tailored response to a Seek job advertisement. 

Once you have set it up, all you will need to do is paste the URL into your notebook, and it will create a tailored letter, highlighting how your experience meets the requirements of the job. 

# Information you need to provide
To start, you need three things.


1.   A secret key from OpenAI: https://platform.openai.com/account/api-keys
2.   A link to a job you would like to apply for: https://www.seek.co.nz/recommended
3.   A public link to a .txt file that outlines your previous experience and qualifications. Drop here and share link: https://drive.google.com/drive/my-drive



In [ ]:
# Go to: https://platform.openai.com/account/api-keys
# Paste new Secret Key here between the quote marks below
api_key = "AddYourInfo"

# Find a relevant job on Seek you would like to apply for: https://www.seek.co.nz/recommended
#Add the url here (remove anything that appears after the question mark '?'):
job_url = "AddYourInfo"

# Paste descriptions of your previous work and qualifications into a .txt file. Drop .txt file onto Google Drive. Click share and share PUBLIC link. 
# Below, replace AddYourInfo with the public URL of your file.
resume_file_url = "AddYourInfo"

# Write your details
your_name = "AddYourInfo"
your_contact_email = "AddYourInfo"
your_contact_number = "AddYourInfo"

# Run all code
## Install libaries

In [ ]:
%%capture
!pip install openai;

In [ ]:
import pandas as pd
import openai
openai.api_key = api_key

# Libraries for parsing data
import requests
from bs4 import BeautifulSoup

# Library for getting your data from Google Doc
from io import StringIO

# Get today's date
import datetime
today = datetime.date.today().strftime("%d %B %Y")

## Scrape Data From Seek Ad

In [ ]:
# OpenAI Embedding Model
GPT_MODEL = "gpt-3.5-turbo"

# Download the text
response = requests.get(job_url, verify=True)
soup = BeautifulSoup(response.text, 'html.parser')
job_text = soup.get_text()
prompt_competencies = "Condense the experience the ideal job candidate should have for this role into 4 or fewer parallel bullets: "


# Get GPT to get the competencies from the text
response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': prompt_competencies},
        {'role': 'user', 'content': job_text},
    ],
    model=GPT_MODEL,
    temperature=0,
)
competencies = response['choices'][0]['message']['content']
number_of_competencies = len(competencies.split('\n'))


# Get Details about the Job from GPT
job_details_prompt = "In bullet points, list ('1 Company:') The company, ('2 The Position:') the position"
response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': job_details_prompt},
        {'role': 'user', 'content': job_text},
    ],
    model=GPT_MODEL,
    temperature=0,
)
job_details = response['choices'][0]['message']['content']


#Print
print(job_details)
print()
print(competencies)


## Read in my CV from a Google doc

In [ ]:
# Get the file ID from the URL

file_id = resume_file_url.split("/")[-2]

# Download the file content as a string
url = f"https://drive.google.com/uc?id={file_id}&export=download"
response = requests.get(url)
resume = StringIO(response.content.decode("utf-8")).read()

print(f"{resume[:1000]}...   truncated for readibility.")
resume

## Combine the Job Ad requirements with my Resume

In [ ]:
my_prompt = f"Here are {number_of_competencies} types of work experience/competencies requested in a job ad. State each competency, then summarise in the first person if I have any relevant experience/skills/competencies (be specific about where the experience was aquired, and list any key details). " 
listed_competencies = f"The {number_of_competencies} competencies to list: {competencies}. Do NOT return any more than the {number_of_competencies} competencies provided."

cover_letter_prompt = my_prompt + listed_competencies# Concatenate the two
response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': cover_letter_prompt},
        {'role': 'user', 'content': resume},
    ],
    model=GPT_MODEL,
    temperature=0,
)

my_competencies = response['choices'][0]['message']['content']
print(my_competencies)

# Write the cover letter

In [ ]:
prompt = f"You are a New Zealand job applicant {your_name}, writing a humble and concise cover letter"
body_instructions = f"Please copy edit the body of a cover letter in first person New Zealand English.\n\
Tone down excitement and confidence. Use bullet points:\n\
Dear Recruitment Team,\n\
Please accept my application for the following role: {job_details}.\n\
The experience and competencies requested for the applicant align with my expertise. Specifically: {my_competencies}.\n\
Please note that I am north island based and would be happy to relocate domestically if relevant. Please feel free to contact me with any questions.\n\
{your_name}\n\
{your_contact_number} \n\
{your_contact_email}"


response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': prompt},
        {'role': 'user', 'content': body_instructions},
    ],
    model=GPT_MODEL,
    temperature=0.7,
)

coverletter_body = response['choices'][0]['message']['content']
print(coverletter_body)

# Ask it to improve the cover letter

In [ ]:
improvements_request = "Radically improve the body of this cover letter by using an example of a great cover letter. "\
                      "Use NZ English but change the structure and reformat to significantly improve it. "\
                      "Do NOT say'excited' or 'confident' or 'ideal' at anytime. "


improve = f"{improvements_request}. # COVER LETTER {coverletter_body}. # JOB BEING APPLIED FOR: {job_text}"
response = openai.ChatCompletion.create(
    messages=[
        {'role': 'user', 'content': improve},
    ],
    model=GPT_MODEL,
    temperature=1,
)

coverletter = response['choices'][0]['message']['content']
print(f"{today}\n\n")
print(coverletter)